In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading in parquet**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@datalakete.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

### **Functions**

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_av_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
select product_id, price, databricks_av_cata.bronze.discount_func(price)as discounted_price from products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_av_cata.bronze.discount_func(price)"))
df.display()


In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_av_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING 
LANGUAGE PYTHON
AS
$$
  return p_brand.upper() 
$$

In [0]:
%sql
select product_id,brand, databricks_av_cata.bronze.upper_func(brand) as brand_upper from products

In [0]:
df_stats = df.groupBy("product_id").agg(sum("price").alias("total_price"))
df_stats.display()

In [0]:
df.write.format("delta")\
   .mode("overwrite")\
   .option("path","abfss://silver@datalakete.dfs.core.windows.net/products")\
    .save()

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_av_cata.silver.products
USING DELTA
LOCATION "abfss://silver@datalakete.dfs.core.windows.net/products"

In [0]:
%sql
select * from databricks_av_cata.silver.products